In [ ]:
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
import pickle
import matplotlib.pyplot as plt

from utils import SEED

In [ ]:
plt.style.use('seaborn-v0_8')

In [ ]:
DATA_DIR = 'data/Movies/'

In [ ]:
ratings = pd.read_csv(DATA_DIR + 'ratings.csv')

In [ ]:
ratings.head()

In [ ]:
movies = pd.read_csv(DATA_DIR + 'movies.csv')

In [ ]:
movies.head()

In [ ]:
movies['movieId'].nunique()

In [ ]:
ratings['rating'].unique()

In [ ]:
movies['year'] = movies['title'].str.extract(r'\((\d{4})\)')

In [ ]:
movies[movies['year'].isna()]

In [ ]:
mean_year = movies[~movies['year'].isna()]['year'].astype(int).mean() 
movies['year'].fillna(value=int(mean_year), inplace=True) 

In [ ]:
year = movies['year'].astype(int).values

In [ ]:
movies

In [ ]:
fig, ax = plt.subplots(figsize=(5, 4))
ax.hist(year)
ax.set_xlabel('Year', fontweight='bold')
ax.set_ylabel('Number of Occurrences', fontweight='bold')
ax.set_title('Year of Movie Release Histogram', fontweight='bold')
plt.tight_layout()

In [ ]:
scaler = MinMaxScaler()

In [ ]:
year_scaled = scaler.fit_transform(year.reshape(-1, 1))

In [ ]:
movies['year'] = year_scaled

In [ ]:
movies['genres'] = movies['genres'].str.split('|')

In [ ]:
movies['genres'].explode().value_counts()

In [ ]:
genres_value_counts = movies['genres'].explode().value_counts()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))
ax.bar(genres_value_counts.index, genres_value_counts.values)
ax.set_xlabel('Genre', fontweight='bold')
ax.set_ylabel('Number of Occurrences', fontweight='bold')
ax.set_title('Genre Histogram', fontweight='bold')
ticks = ax.get_xticklabels()
ax.set_xticks(
    [t.get_position()[0] for t in ticks], 
    [t.get_text() for t in ticks],
    rotation=90
)
plt.tight_layout()

In [ ]:
vectorizer = CountVectorizer()

In [ ]:
def get_ingredient(row):
    for i, v in enumerate(row):
        row[i] = v.replace(' ', '_').replace('-', '_')
    return ' '.join(row)

genres = movies['genres'].apply(get_ingredient)

In [ ]:
genres_encoded = vectorizer.fit_transform(genres)

In [ ]:
vectorizer.get_feature_names_out()

In [ ]:
genres_df = pd.DataFrame(columns=vectorizer.get_feature_names_out(), data=genres_encoded.toarray())

In [ ]:
genres_df.shape

In [ ]:
movies_processed = movies.loc[:, ['movieId', 'year']]

In [ ]:
movies_processed = pd.concat((movies_processed, genres_df), axis=1)

In [ ]:
ratings_sample = ratings[ratings['rating'] >= 4].sample(200000)

In [ ]:
common_items_ids = np.intersect1d(
    np.unique(ratings_sample['movieId'].values), 
    np.unique(movies_processed['movieId'].values)
    )

In [ ]:
movies_processed = movies_processed[movies_processed['movieId'].isin(common_items_ids)]
ratings_sample = ratings_sample[ratings_sample['movieId'].isin(common_items_ids)]

In [ ]:
ratings_sample['movieId'].unique().shape

In [ ]:
ratings_sample['userId'].unique().shape

In [ ]:
user_mapping = {userid: i for i, userid in enumerate(ratings_sample['userId'].unique())}
item_mapping = {movieid: i for i, movieid in enumerate(movies_processed['movieId'].unique())}

In [ ]:
user_ids = torch.LongTensor([user_mapping[i] for i in ratings_sample['userId']])
item_ids = torch.LongTensor([item_mapping[i] for i in ratings_sample['movieId']])
edge_index = torch.stack((user_ids, item_ids))

In [ ]:
train_index, val_index = train_test_split(range(len(ratings_sample)), test_size=0.25, random_state=SEED)

train_edge_index = edge_index[:, train_index]
val_edge_index = edge_index[:, val_index]

In [ ]:
movies_processed.loc[:, 'ID'] = movies_processed.loc[:, 'movieId'].apply(lambda i: item_mapping[i])
movies_processed.sort_values(by='movieId', inplace=True)

In [ ]:
items_features = torch.Tensor(movies_processed.drop(columns=['movieId', 'ID']).values)

In [ ]:
users_features = torch.Tensor(np.zeros(ratings_sample.shape[0]).reshape(-1, 1))

In [ ]:
movies_dataset = {
    'users_features': users_features,
    'items_features': items_features,
    'train_edge_index': train_edge_index,
    'val_edge_index': val_edge_index
}

In [ ]:
with open('datasets/movies_dataset.bin', 'wb') as f:
    pickle.dump(movies_dataset, f)